In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib
import os

AUTOTUNE = tf.data.experimental.AUTOTUNE
tf.keras.backend.clear_session()  # For easy reset of notebook state.

tf.__version__
tf.executing_eagerly()


True

In [2]:

ds = tfds.load(name="imagenette", with_info=True, split=["train","validation"])
ds_train=ds[0][0]
ds_test=ds[0][1]
assert isinstance(ds_train, tf.data.Dataset)

def normalize(x):
    y = {'image': tf.image.convert_image_dtype(x['image'], tf.float32), 'label': x['label']}
    y = (tf.image.resize(y['image'], (224,224)), y['label'])
    return y
    
ds_train = ds_train.map(lambda x: normalize(x))
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(100)
ds_train = ds_train.batch(8)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(8)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# VGG 11

In [3]:
inputs = keras.Input(shape=(224,224,3))

x = layers.Conv2D(64, 3, activation='relu')(inputs)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.Conv2D(512, 3, activation='relu')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(4096, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

VGG11 = keras.Model(inputs, outputs)
VGG11.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 256)       590080

In [4]:
#keras.utils.plot_model(VGG11, 'my_first_model.png')


In [5]:
VGG11.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)


In [ ]:
VGG11.fit(
    x=ds_train,
    #steps_per_epoch=1,
    epochs=3,
    validation_data=ds_test,
)

Epoch 1/3
     48/Unknown - 233s 5s/step - loss: 3.3279 - accuracy: 0.1250

# VGG-16

In [29]:
inputs = keras.Input(shape=(224,224,3))

x = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.Conv2D(512, 3, activation='relu', padding='same')(x)
x = layers.MaxPool2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(4096, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)

VGG11 = keras.Model(inputs, outputs)
VGG11.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_85 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_86 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 56, 56, 128)       0   

# Keras VGG 16

In [23]:
VGG_keras = keras.applications.VGG16()

In [24]:
VGG_keras.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     